<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Natural%20Language%20Processing/Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in Tensorflow

In [1]:
!nvidia-smi

Thu Feb 16 18:09:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    24W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-16 18:09:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-16 18:09:40 (106 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import plot_loss_curves,unzip_data,create_tensorboard_callback,compare_historys

## Getting a text Dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-16 18:09:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.145.128, 173.194.69.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-02-16 18:09:43 (115 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualizing Dataset

In [6]:
import pandas as pd
train_dataset = pd.read_csv("/content/train.csv")

In [7]:
train_dataset

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
train_dataset_shuffled = train_dataset.sample(frac = 1,random_state = 42)


In [9]:
train_dataset_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [10]:
test_data  = pd.read_csv("/content/test.csv")

In [11]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [12]:
train_dataset.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
import random
random_index = random.randint(0,len(train_dataset)-5)
for row in train_dataset_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(text,target)

Severe Thunderstorm Warning including Midwest City OK Del City OK Choctaw OK until 10:30 PM CDT http://t.co/ogxSY4GWD1 1
Kach was a group to which belonged Baruch Goldstein a mass murderer who in 1994 shot and killed 29 PalestinianÛ_ http://t.co/bXGNQ57xvb 1
Rly tragedy in MP: Some live to recount horror: ÛÏWhen I saw coaches of my train plunging into water I called ... http://t.co/CaR5QEUVHH 1
Criminals Who Hijack Lorries And Buses Arrested In Enugu (PHOTO) http://t.co/LRTU8Rwn2f 1
@Jannet2208 I fell off someone's back and hit my head on concrete /: I was bleeding n shit 0


## Split Data into Training and Validation Sets

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences,val_sentences,train_labels,val_labels = train_test_split(
    train_dataset_shuffled['text'].to_numpy(),
    train_dataset_shuffled['target'].to_numpy(),
    test_size = 0.1,
    random_state = 42
    )

In [16]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

## Tokenization and Embedding

### Text Vectorization (Tokenization)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = 10000,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None,
    pad_to_max_tokens = True,
  )


In [18]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

14.901036345059115

### Text Vecotrizer Variables

In [19]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)

In [20]:
text_vectorizer.adapt(train_sentences)

In [21]:
sample_sentence = "Flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
random_sentences = random.choice(train_sentences)
text_vectorizer([random_sentences])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,  565,   10, 3716,    2, 3837, 6960,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

## Embedding Layer

In [24]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
      output_dim = 128,
      input_length = max_length
                                   
)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
embedded_sentence = embedding(text_vectorizer(random_sentence))
print(embedded_sentence)

@USAgov Koreans are performing hijacking of the Tokyo Olympic Games.https://t.co/APkSnpLXZj
tf.Tensor(
[[ 0.00360226 -0.02527996  0.03172812 ...  0.02124466 -0.01975948
  -0.00828673]
 [-0.0365895  -0.01421992  0.02080803 ... -0.04508554 -0.03730401
   0.01521916]
 [-0.0051994   0.04077441 -0.02807071 ... -0.01232469  0.00121338
  -0.02996054]
 ...
 [ 0.04253422 -0.04661579  0.04137712 ...  0.03347408  0.03761412
   0.01891274]
 [ 0.04253422 -0.04661579  0.04137712 ...  0.03347408  0.03761412
   0.01891274]
 [ 0.04253422 -0.04661579  0.04137712 ...  0.03347408  0.03761412
   0.01891274]], shape=(15, 128), dtype=float32)


## Model 0 : Naive Bayes (Baseline)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers,
    ("clf",MultinomialNB()), #model the text,
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
base_line_score = model_0.score(val_sentences,val_labels)

In [28]:
base_line_score

0.7926509186351706

In [29]:
base_line_predict = model_0.predict(val_sentences)
base_line_predict[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating functions to compare model experiments

In [30]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = "weighted")
  model_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_results

In [31]:
base_line_results = calculate_results(val_labels,base_line_predict)
base_line_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: Simple Dense Model

In [32]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [33]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [35]:
model_1.compile(loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [36]:
model_1_history = model_1.fit(
    x = train_sentences,y = train_labels,
    epochs = 5,
    validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_1_dense")]
    )

Saving TensorBoard log files to: model_logs/model_1_dense/20230216-180948
Epoch 1/5
215/215 [==============================] - 21s 74ms/step - loss: 0.6396 - accuracy: 0.6462 - val_loss: 0.5737 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.4699 - accuracy: 0.8323 - val_loss: 0.4723 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3400 - accuracy: 0.8770 - val_loss: 0.4501 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2569 - accuracy: 0.9082 - val_loss: 0.4530 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1973 - accuracy: 0.9350 - val_loss: 0.4643 - val_accuracy: 0.7940


In [37]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4643 - accuracy: 0.7940


[0.46426135301589966, 0.7939632534980774]

In [38]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [39]:
model_1_pred_probs[10]

array([0.20111609], dtype=float32)

In [40]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [41]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [42]:
model_1_results = calculate_results(y_true = val_labels,y_pred = model_1_preds)
model_1_results

{'accuracy': 79.39632545931758,
 'precision': 0.7981039765787369,
 'recall': 0.7939632545931758,
 'f1': 0.7914089039180439}

## Visualizing Learned Embeddings

In [43]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [44]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [45]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]

In [46]:
embed_weights.shape

(10000, 128)

In [47]:
embed_weights

array([[ 0.21514963, -0.02694481, -0.05261353, ..., -0.0099529 ,
        -0.07427995, -0.06683377],
       [-0.0365895 , -0.02642009, -0.04623136, ..., -0.04508554,
        -0.07446823, -0.06481213],
       [-0.03116828, -0.02142205, -0.06051754, ..., -0.02030781,
        -0.04478261, -0.03602117],
       ...,
       [-0.02442099, -0.03852652,  0.01564867, ...,  0.04180614,
        -0.00398181, -0.00853052],
       [ 0.00799213, -0.01117676, -0.04417386, ..., -0.00954732,
        -0.01990587, -0.04569342],
       [ 0.02198018,  0.0789622 , -0.03906866, ...,  0.06689966,
        -0.02405645, -0.01710626]], dtype=float32)

## Recurrent Neural Network

### Model 2: LSTM

In [48]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences = True)(x) # When we are stacking we need return_sequences = True
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [49]:
model_2.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [50]:
model_2_history = model_2.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_2_lstm")]
            )

Saving TensorBoard log files to: model_logs/model_2_lstm/20230216-181031
Epoch 1/5
215/215 [==============================] - 15s 44ms/step - loss: 0.3797 - accuracy: 0.8348 - val_loss: 0.4536 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2469 - accuracy: 0.8994 - val_loss: 0.5217 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1753 - accuracy: 0.9326 - val_loss: 0.5638 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1304 - accuracy: 0.9511 - val_loss: 0.7586 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0933 - accuracy: 0.9638 - val_loss: 1.0093 - val_accuracy: 0.7795


In [51]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[9.0470398e-03],
       [4.6809390e-01],
       [9.9971205e-01],
       [1.5781872e-02],
       [2.1479085e-04],
       [9.9991798e-01],
       [9.1269785e-01],
       [9.9997759e-01],
       [9.9997103e-01],
       [2.0575535e-01]], dtype=float32)

In [52]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [53]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [54]:
calculate_results(val_labels,model_2_preds)

{'accuracy': 77.95275590551181,
 'precision': 0.7813761276529175,
 'recall': 0.7795275590551181,
 'f1': 0.7775281328700309}

### Model-3 GRU Powered RNN

In [55]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#If returns sequences and stackings are true
# x = layers.LSTM(64,return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64,activation = "relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [56]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [57]:
model_3.compile(loss = "binary_crossentropy",metrics = ['accuracy'],optimizer = "adam")

In [58]:
model_3_history = model_3.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_3_gru")]
            )

Saving TensorBoard log files to: model_logs/model_3_gru/20230216-181058
Epoch 1/5
215/215 [==============================] - 11s 40ms/step - loss: 0.2085 - accuracy: 0.9130 - val_loss: 0.7083 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1002 - accuracy: 0.9645 - val_loss: 0.8047 - val_accuracy: 0.7598
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0791 - accuracy: 0.9718 - val_loss: 0.8191 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0663 - accuracy: 0.9742 - val_loss: 1.0009 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0532 - accuracy: 0.9781 - val_loss: 1.1621 - val_accuracy: 0.7625


In [59]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[5.46893571e-04],
       [8.51516962e-01],
       [9.99500513e-01],
       [7.74607286e-02],
       [2.50619836e-04],
       [9.99838948e-01],
       [1.11478746e-01],
       [9.99880672e-01],
       [9.99910116e-01],
       [9.98890460e-01]], dtype=float32)

In [60]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))

In [61]:
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [62]:
calculate_results(model_3_pred,val_labels)

{'accuracy': 76.24671916010499,
 'precision': 0.7686227970878954,
 'recall': 0.7624671916010499,
 'f1': 0.7639391677278325}

### Model-4 Bidirectional RNN

In [63]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1,activation = "sigmoid",)(x)
model_4 = tf.keras.Model(inputs,outputs)

In [64]:
model_4.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                             

In [65]:
model_4.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [66]:
mode_4_history = model_4.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_4_bidirectional")],
)

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230216-181120
Epoch 1/5
215/215 [==============================] - 19s 57ms/step - loss: 0.1182 - accuracy: 0.9495 - val_loss: 1.0801 - val_accuracy: 0.7428
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0587 - accuracy: 0.9739 - val_loss: 1.2423 - val_accuracy: 0.7520
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0483 - accuracy: 0.9780 - val_loss: 1.8962 - val_accuracy: 0.7533
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0449 - accuracy: 0.9790 - val_loss: 1.4219 - val_accuracy: 0.7493
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0429 - accuracy: 0.9790 - val_loss: 1.4788 - val_accuracy: 0.7507


In [67]:
model_4_preds = tf.squeeze(tf.round(model_4.predict(val_sentences)))

24/24 [==============================] - 1s 5ms/step


In [68]:
calculate_results(val_labels,model_4_preds)

{'accuracy': 75.06561679790026,
 'precision': 0.7502185026167505,
 'recall': 0.7506561679790026,
 'f1': 0.7502499687539057}

### Model-5: Conv1D 

In [82]:
embedding_text = embedding(text_vectorizer(['this is a test sentence for embedding']))

In [83]:
embedding_text

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01108818,  0.00038096, -0.06977408, ..., -0.00097736,
         -0.09755569, -0.07064315],
        [-0.02156954, -0.02264068, -0.03645947, ..., -0.00126786,
         -0.03841352, -0.01440422],
        [-0.04161504, -0.00856694, -0.05443833, ..., -0.00481963,
         -0.02469284, -0.03355973],
        ...,
        [ 0.1505231 , -0.02668592, -0.0319179 , ..., -0.01524423,
         -0.05472673, -0.03214425],
        [ 0.1505231 , -0.02668592, -0.0319179 , ..., -0.01524423,
         -0.05472673, -0.03214425],
        [ 0.1505231 , -0.02668592, -0.0319179 , ..., -0.01524423,
         -0.05472673, -0.03214425]]], dtype=float32)>

In [91]:
conv_1d = layers.Conv1D(filters = 32,
kernel_size = 5,
activation = "relu",
padding = "same"
)

In [92]:
conv_1d_output = conv_1d(embedding_text)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_text.shape, conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [93]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.01769976, 0.0969847 , 0.08928566, 0.12451448, 0.0365254 ,
        0.0404409 , 0.03976125, 0.05706856, 0.04041123, 0.03910651,
        0.03727398, 0.00797712, 0.0296589 , 0.19657941, 0.012976  ,
        0.12031446, 0.01679485, 0.08640224, 0.07545591, 0.03424371,
        0.08705965, 0.09384744, 0.        , 0.05467544, 0.10836939,
        0.10342743, 0.0685901 , 0.11739205, 0.05592701, 0.        ,
        0.0429213 , 0.12811455]], dtype=float32)>